Tokenization

In [ ]:
# -------------------------------------------------
# TinyStories-10k + BPE Tokenizer (All-in-One Cell)
# -------------------------------------------------

import json
import re
import os
import random
from collections import Counter, defaultdict


# ===========================================
# STEP 1 — Generate TinyStories-10k IN COLAB
# ===========================================

def generate_story():
    subjects = [
        "a little dragon", "a young wizard", "a brave robot", "a curious child",
        "a tiny bear", "a happy fairy", "a small puppy", "a gentle giant",
        "a clever cat", "a playful elf"
    ]

    actions = [
        "found", "lost", "built", "discovered", "met", "followed", "searched for",
        "protected", "explored", "dreamed about"
    ]

    objects = [
        "a magic book", "a glowing key", "a secret door", "a hidden map",
        "a tiny spaceship", "an ancient scroll", "a shiny crystal",
        "a mysterious potion", "a floating island", "a talking tree"
    ]

    endings = [
        "and learned something new.",
        "and became very brave.",
        "and made a new friend.",
        "and felt very happy.",
        "and went on an adventure.",
        "and found the meaning of courage.",
        "and discovered a new world.",
        "and shared the story with everyone.",
        "which changed their life forever.",
        "but the journey had just begun."
    ]

    story = f"Once upon a time, {random.choice(subjects)} {random.choice(actions)} {random.choice(objects)} {random.choice(endings)}"
    return story


# Generate 10,000 synthetic TinyStories lines
stories = [generate_story() for _ in range(10000)]

with open("tinystories_10k.txt", "w", encoding="utf-8") as f:
    for s in stories:
        f.write(s + "\n")

print("TinyStories-10k generated. Lines:", len(stories))


# ===========================================
# STEP 2 — BPE Tokenizer
# ===========================================

# -------------------------------------------------
# tokenizer.py — FIXED and stable BPE Tokenizer
# -------------------------------------------------

import json
import re
import os
from collections import Counter, defaultdict

class BPETokenizer:
    def __init__(self, vocab_size=5000):
        self.vocab_size = vocab_size
        self.word_freqs = Counter()
        self.bpe_merges = []
        self.vocab = {}
        self.inv_vocab = {}

    # -----------------------------------------------------
    # Train BPE tokenizer
    # -----------------------------------------------------
    def train(self, text):
        words = text.split()

        # Count words in character form
        for w in words:
            tokens = list(w) + ["</w>"]
            self.word_freqs[tuple(tokens)] += 1

        # Perform merges until we reach vocab_size
        while len(self.bpe_merges) < (self.vocab_size - 256):   # leaving room for chars
            pairs = self._get_pair_counts()
            if not pairs:
                break

            best_pair = max(pairs, key=pairs.get)
            self.bpe_merges.append(best_pair)
            self._merge_pair(best_pair)

            if len(self.bpe_merges) % 200 == 0:
                print("Merges:", len(self.bpe_merges))

        # Build final vocab from:
        # - all characters seen
        # - all merged pairs
        vocab = set()

        # characters
        for word in self.word_freqs:
            for tok in word:
                vocab.add(tok)

        # merged tokens
        for a, b in self.bpe_merges:
            vocab.add(a + b)

        vocab.add("</w>")  # ensure always present

        vocab = sorted(list(vocab))
        self.vocab = {tok: i for i, tok in enumerate(vocab)}
        self.inv_vocab = {i: tok for tok, i in self.vocab.items()}

        print("Final vocab size:", len(self.vocab))

    # -----------------------------------------------------
    def _get_pair_counts(self):
        pairs = Counter()
        for word, freq in self.word_freqs.items():
            syms = list(word)
            for i in range(len(syms) - 1):
                pairs[(syms[i], syms[i+1])] += freq
        return pairs

    # -----------------------------------------------------
    def _merge_pair(self, pair):
        new_freqs = Counter()
        bigram = " ".join(pair)
        pat = re.compile(re.escape(bigram))

        for word, freq in self.word_freqs.items():
            w = " ".join(word)
            w_new = pat.sub("".join(pair), w)
            new_freqs[tuple(w_new.split())] += freq

        self.word_freqs = new_freqs

    # -----------------------------------------------------
    def _apply_bpe(self, tokens):
        merges_set = set(tuple(m) for m in self.bpe_merges)
        changed = True

        while changed:
            changed = False
            i = 0
            while i < len(tokens)-1:
                if (tokens[i], tokens[i+1]) in merges_set:
                    tokens = tokens[:i] + [tokens[i] + tokens[i+1]] + tokens[i+2:]
                    changed = True
                else:
                    i += 1
        return tokens

    # -----------------------------------------------------
    def encode(self, text):
        ids = []
        for w in text.split():
            tokens = self._apply_bpe(list(w) + ["</w>"])
            for t in tokens:
                ids.append(self.vocab.get(t, self.vocab["</w>"]))
        return ids

    # -----------------------------------------------------
    def decode(self, ids):
        toks = [self.inv_vocab[i] for i in ids]
        text = "".join(toks)
        return text.replace("</w>", " ").strip()

    # -----------------------------------------------------
    def save(self, folder="bpe_tokenizer"):
        os.makedirs(folder, exist_ok=True)
        json.dump(self.vocab, open(f"{folder}/vocab.json", "w"))
        json.dump(self.bpe_merges, open(f"{folder}/merges.json", "w"))

    @staticmethod
    def load(folder="bpe_tokenizer"):
        tok = BPETokenizer()
        tok.vocab = json.load(open(f"{folder}/vocab.json"))
        tok.inv_vocab = {v: k for k, v in tok.vocab.items()}
        tok.bpe_merges = json.load(open(f"{folder}/merges.json"))
        return tok



# ===========================================
# STEP 3 — Train and save tokenizer
# ===========================================

text = open("tinystories_10k.txt", "r", encoding="utf-8").read()

tokenizer = BPETokenizer(vocab_size=5000)
tokenizer.train(text)
tokenizer.save("bpe_tokenizer")

print("Tokenizer saved successfully!")


TinyStories-10k generated. Lines: 10000
Merges: 200
Final vocab size: 275
Tokenizer saved successfully!


Data Preparation

In [ ]:
# ------------------------------
# GoLab Cell 2 — Dataset Preparation
# ------------------------------

import os
import torch
from pathlib import Path

# parameters (tweak if you want)
batch_size = 64
block_size = 256   # context length for model
val_ratio = 0.1
seed = 1337
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(seed)

# ---- 1) Paths ----
dataset_path = "tinystories_10k.txt"
tokenizer_folder = "bpe_tokenizer"
save_dir = "bin"
os.makedirs(save_dir, exist_ok=True)

# ---- 2) Load raw text ----
assert Path(dataset_path).exists(), f"Dataset not found: {dataset_path}"
with open(dataset_path, "r", encoding="utf-8") as f:
    text = f.read()
print("Loaded dataset chars:", len(text))

# ---- 3) Load tokenizer ----
# We assume BPETokenizer class is defined in the notebook (tokenizer cell).
tokenizer = BPETokenizer.load(tokenizer_folder)
print("Loaded tokenizer, vocab size:", len(tokenizer.vocab))

# ---- 4) Encode entire corpus to integer IDs ----
# Note: tokenizer.encode works on strings (splits on whitespace internally)
all_ids = tokenizer.encode(text)
data = torch.tensor(all_ids, dtype=torch.long)
print("Total tokens:", data.size(0))

# ---- 5) Train / Val split ----
n = int((1 - val_ratio) * len(data))
train_data = data[:n].clone()
val_data = data[n:].clone()
print(f"Train tokens: {train_data.size(0)}, Val tokens: {val_data.size(0)}")

# ---- 6) Save tensors to disk ----
torch.save(train_data, os.path.join(save_dir, "train.pt"))
torch.save(val_data, os.path.join(save_dir, "val.pt"))
print("Saved train/val tensors to 'bin/'")

# ---- 7) Basic get_batch function (returns x,y on device) ----
def get_batch(split):
    """
    Returns:
      x, y: LongTensors of shape (batch_size, block_size)
      x = input tokens, y = target tokens (shifted by one)
    """
    data_src = train_data if split == 'train' else val_data
    # pick random starting indices
    ix = torch.randint(0, len(data_src) - block_size - 1, (batch_size,))
    x = torch.stack([data_src[i:i+block_size] for i in ix])
    y = torch.stack([data_src[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

# ---- 8) Sanity checks (decode a small slice) ----
sample_index = 0
sample_slice = train_data[sample_index: sample_index + 60].tolist()
print("Sample token ids (first 60):", sample_slice[:30])
try:
    decoded = tokenizer.decode(sample_slice)
    print("Decoded sample:", decoded[:200])
except Exception as e:
    print("Decode failed (ok for some BPE id layouts):", e)

# ---- done ----
print("Dataset prepared — ready to train. Call get_batch('train') to get batches.")


Loaded dataset chars: 852012
Loaded tokenizer, vocab size: 275
Total tokens: 230926
Train tokens: 207833, Val tokens: 23093
Saved train/val tensors to 'bin/'
Sample token ids (first 60): [4, 254, 5, 247, 5, 34, 1, 207, 152, 231, 1, 5, 238, 252, 41, 240, 1, 138, 121, 139, 231, 1, 28, 4, 254, 5, 247, 5, 273, 266]
Decoded sample: Once upon a time, a brave robot lost a talking tree but the journey had just begun. Once upon a time, a young wizard discovered an ancient  cr ll and  hared the story with everyone. Once upon a time, 
Dataset prepared — ready to train. Call get_batch('train') to get batches.


Model

In [ ]:
# models/model.py
# NanoGPT-A2 — minimal GPT-style model (PyTorch)
# Clean, readable, research-friendly, and ready for Layer-5 hooks.

import math
import torch
import torch.nn as nn
from torch.nn import functional as F

class GPTConfig:
    """Minimal config container."""
    def __init__(self, vocab_size, block_size,
                 n_layer=6, n_head=6, n_embd=384, dropout=0.2):
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_layer = n_layer
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout

# -------------------------
# Attention head (single)
# -------------------------
class Head(nn.Module):
    def __init__(self, config, head_size):
        super().__init__()
        n_embd = config.n_embd
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # causal mask: registered as buffer so it moves with model.to(device)
        self.register_buffer("tril", torch.tril(torch.ones(config.block_size, config.block_size)))
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        # x: (B, T, C)
        B, T, C = x.size()
        k = self.key(x)    # (B, T, hs)
        q = self.query(x)  # (B, T, hs)
        # compute attention scores
        wei = q @ k.transpose(-2, -1) * (k.size(-1) ** -0.5)  # (B, T, T)
        # causal masking (prevent attending to future)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)  # (B, T, hs)
        out = wei @ v      # (B, T, hs)
        return out

# -------------------------
# Multi-head attention
# -------------------------
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        head_size = config.n_embd // config.n_head
        self.heads = nn.ModuleList([Head(config, head_size) for _ in range(config.n_head)])
        self.proj = nn.Linear(config.n_embd, config.n_embd)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)  # (B, T, C)
        out = self.proj(out)
        out = self.dropout(out)
        return out

# -------------------------
# Feed-forward network (MLP)
# -------------------------
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.dropout),
        )

    def forward(self, x):
        return self.net(x)

# -------------------------
# Transformer block
# -------------------------
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.attn = MultiHeadAttention(config)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.mlp = FeedForward(config)

    def forward(self, x):
        # --- attention (with residual) ---
        x = x + self.attn(self.ln1(x))
        # --- MLP (with residual) ---
        x = x + self.mlp(self.ln2(x))
        return x

# -------------------------
# Full GPT language model
# -------------------------
class GPTLanguageModel(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.config = config

        # token and positional embeddings
        self.tok_emb = nn.Embedding(config.vocab_size, config.n_embd)
        self.pos_emb = nn.Embedding(config.block_size, config.n_embd)

        # stack of transformer blocks
        self.blocks = nn.ModuleList([Block(config) for _ in range(config.n_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd)  # final layer norm

        # language modeling head (tie weights optionally)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # initialize weights
        self.apply(self._init_weights)

        # tie weights like GPT-2: lm_head weight = tok_emb weight
        self.lm_head.weight = self.tok_emb.weight

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
        if isinstance(module, nn.Linear) and module.bias is not None:
            nn.init.zeros_(module.bias)
        if isinstance(module, nn.LayerNorm):
            nn.init.zeros_(module.bias)
            nn.init.ones_(module.weight)

    def forward(self, idx, targets=None):
        """
        idx: (B, T) token indices
        targets: (B, T) token indices (optional)
        returns: logits (B, T, V), loss (scalar) if targets provided
        """
        device = idx.device
        B, T = idx.size()
        assert T <= self.config.block_size, f"Sequence length {T} > block_size {self.config.block_size}"

        # token + position embeddings
        tok_emb = self.tok_emb(idx)                       # (B, T, C)
        pos = torch.arange(T, device=device)
        pos_emb = self.pos_emb(pos)                       # (T, C)
        x = tok_emb + pos_emb                              # (B, T, C)

        # --- Optionally: capture residual stream after embeddings for Layer-5 hooks ---
        # e.g., residual = x.clone() or call registered hooks here

        # transformer blocks
        for block in self.blocks:
            x = block(x)

        x = self.ln_f(x)                                   # (B, T, C)
        logits = self.lm_head(x)                           # (B, T, V)

        loss = None
        if targets is not None:
            # reshape for cross entropy
            loss = F.cross_entropy(logits.view(B*T, -1), targets.view(B*T))

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Auto-regressive generation.
        idx: (B, T) starting context
        returns: (B, T + max_new_tokens)
        """
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.config.block_size:]  # crop to block_size
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature     # (B, V)

            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                min_topk = v[:, -1].unsqueeze(1)
                logits = torch.where(logits < min_topk, torch.full_like(logits, -1e10), logits)

            probs = F.softmax(logits, dim=-1)            # (B, V)
            next_token = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat([idx, next_token], dim=1)             # (B, T+1)

        return idx

# -------------------------
# Utility: count parameters
# -------------------------
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())/1e6

# -------------------------
# Quick smoke test when run as script
# -------------------------
if __name__ == "__main__":
    # small test to validate shapes
    cfg = GPTConfig(vocab_size=1000, block_size=64, n_layer=4, n_head=4, n_embd=128, dropout=0.1)
    m = GPTLanguageModel(cfg)
    x = torch.randint(0, cfg.vocab_size, (2, 32))
    logits, loss = m(x, targets=x)
    print("logits shape:", logits.shape, "loss:", loss.item())
    print("Params (M):", count_parameters(m))


logits shape: torch.Size([2, 32, 1000]) loss: 6.535799503326416
Params (M): 0.928


Training

In [ ]:
# ---------- TRAIN.PY (COLAB VERSION) ----------
# Assumes:
# - BPETokenizer class is already defined in notebook
# - GPTConfig and GPTLanguageModel are already defined in notebook
# - train.pt / val.pt created

import torch
import time

# ---------------------
# Hyperparameters
# ---------------------
batch_size = 64
block_size = 256
learning_rate = 3e-4
max_iters = 1500
eval_interval = 200
eval_iters = 100
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(1337)

# ---------------------
# Load dataset
# ---------------------
train_data = torch.load("bin/train.pt")
val_data   = torch.load("bin/val.pt")

# ---------------------
# Load tokenizer
# ---------------------
tokenizer = BPETokenizer.load("bpe_tokenizer")
vocab_size = len(tokenizer.vocab)

print("Vocab size:", vocab_size)
print("Train tokens:", len(train_data))
print("Val tokens:", len(val_data))

# ---------------------
# get_batch function
# ---------------------
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

# ---------------------
# Evaluation
# ---------------------
@torch.no_grad()
def estimate_loss(model):
    model.eval()
    out = {}

    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()

    model.train()
    return out

# ---------------------
# Build model
# ---------------------
config = GPTConfig(
    vocab_size=vocab_size,
    block_size=block_size,
    n_layer=6,
    n_head=6,
    n_embd=384,
    dropout=0.2,
)

model = GPTLanguageModel(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

print(f"Model parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

# ---------------------
# Training loop
# ---------------------
for it in range(max_iters):

    if it % eval_interval == 0 or it == max_iters - 1:
        losses = estimate_loss(model)
        print(f"Step {it}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # save checkpoint
        torch.save(
            {
                'model': model.state_dict(),
                'config': config.__dict__,
            },
            "checkpoint.pt"
        )
        print("Checkpoint saved.")

    xb, yb = get_batch('train')

    _, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("Training complete.")


Vocab size: 275
Train tokens: 207833
Val tokens: 23093
Model parameters: 10.84M
Step 0: train loss 5.7343, val loss 5.7345
Checkpoint saved.
Step 200: train loss 0.5002, val loss 0.5010
Checkpoint saved.
Step 400: train loss 0.4181, val loss 0.4199
Checkpoint saved.
Step 600: train loss 0.4067, val loss 0.4094
Checkpoint saved.
Step 800: train loss 0.4062, val loss 0.4085
Checkpoint saved.
Step 1000: train loss 0.4033, val loss 0.4070
Checkpoint saved.
Step 1200: train loss 0.4086, val loss 0.4123
Checkpoint saved.
Step 1400: train loss 0.4030, val loss 0.4071
Checkpoint saved.
Step 1499: train loss 0.4020, val loss 0.4061
Checkpoint saved.
Training complete.


Baseline Evaluation

In [ ]:
# ============================================================
# CLEAN BASELINE EVALUATION FOR NANOGPT-A2 (COLAB VERSION)
# ============================================================

import os, json, math, torch
from tqdm import trange

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


# ============================================================
# 1. TOKENIZER (already defined earlier in Colab)
# ============================================================
if not os.path.exists("bpe_tokenizer/vocab.json"):
    raise FileNotFoundError("Tokenizer not found.")

tokenizer = BPETokenizer.load("bpe_tokenizer")
print("Tokenizer loaded. Vocab size:", len(tokenizer.vocab))


# ============================================================
# 2. LOAD TRAIN + VAL DATA
# ============================================================
train_data = torch.load("bin/train.pt")
val_data   = torch.load("bin/val.pt")
print("Train tokens:", len(train_data))
print("Val tokens  :", len(val_data))


# ============================================================
# 3. BUILD MODEL (already defined earlier in Colab)
# ============================================================
config = GPTConfig(
    vocab_size=len(tokenizer.vocab),
    block_size=256,
    n_layer=6,
    n_head=6,
    n_embd=384,
    dropout=0.2
)

model = GPTLanguageModel(config).to(device)

if not os.path.exists("checkpoint.pt"):
    raise FileNotFoundError("checkpoint.pt missing.")

ckpt = torch.load("checkpoint.pt", map_location=device)
model.load_state_dict(ckpt["model"])
print("✓ Model restored from checkpoint.")


# ============================================================
# 4. GET BATCH + PERPLEXITY
# ============================================================
def get_batch(data_tensor, batch_size=32, block_size=256):
    ix = torch.randint(0, len(data_tensor) - block_size - 1, (batch_size,))
    x = torch.stack([data_tensor[i:i+block_size] for i in ix])
    y = torch.stack([data_tensor[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)


@torch.no_grad()
def compute_perplexity(model, data_tensor, n_iter=50, batch_size=32):
    model.eval()
    losses = []
    for _ in range(n_iter):
        xb, yb = get_batch(data_tensor, batch_size, config.block_size)
        _, loss = model(xb, yb)
        losses.append(loss.item())
    model.train()
    loss = sum(losses) / len(losses)
    return loss, math.exp(loss)


# ============================================================
# 5. STREAMING STATS PER LAYER
# ============================================================
class RunningStats:
    def __init__(self, C):
        self.count = 0
        self.mean = torch.zeros(C)
        self.M2 = torch.zeros(C)

    def update(self, x):
        x = x.reshape(-1, x.size(-1)).detach().cpu()
        for row in x:
            self.count += 1
            delta = row - self.mean
            self.mean += delta / self.count
            delta2 = row - self.mean
            self.M2 += delta * delta2

    def finalize(self):
        var = self.M2 / max(1, self.count - 1)
        return {
            "mean": self.mean.tolist(),
            "std": torch.sqrt(var).tolist(),
            "mean_norm": float(self.mean.norm().item())
        }


def collect_streaming_stats(model, data_tensor, n_batches=50, batch_size=32):
    C = model.config.n_embd
    stats = {i: RunningStats(C) for i in range(model.config.n_layer)}

    hooks = []
    def hook_factory(layer):
        def hook(module, inp, out):
            stats[layer].update(out)
        return hook

    for i, block in enumerate(model.blocks):
        hooks.append(block.register_forward_hook(hook_factory(i)))

    for _ in trange(n_batches):
        xb, _ = get_batch(data_tensor, batch_size, config.block_size)
        model(xb)

    for h in hooks:
        h.remove()

    return {i: stats[i].finalize() for i in stats}


# ============================================================
# 6. JSON-SAFE CONVERSION
# ============================================================
def to_python(obj):
    """Recursively convert tensors → lists/floats for JSON."""
    if isinstance(obj, torch.Tensor):
        if obj.dim() == 0:
            return obj.item()
        return obj.tolist()
    if isinstance(obj, dict):
        return {k: to_python(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [to_python(x) for x in obj]
    return obj


# ============================================================
# 7. MAIN EVAL
# ============================================================
print("\n========== Computing Perplexities ==========")
train_loss, train_ppl = compute_perplexity(model, train_data)
val_loss, val_ppl = compute_perplexity(model, val_data)

print("Train Loss:", train_loss, "Train PPL:", train_ppl)
print("Val Loss :", val_loss, "Val PPL :", val_ppl)

print("\n========== Collecting Layer Stats ==========")
stats_train = collect_streaming_stats(model, train_data)
stats_val   = collect_streaming_stats(model, val_data)

print("\n========== Computing Drift ==========")
drift = {}
for layer in stats_train:
    m1 = torch.tensor(stats_train[layer]["mean"])
    m2 = torch.tensor(stats_val[layer]["mean"])
    l2 = float(torch.norm(m1 - m2))
    rel = l2 / (torch.norm(m1) + 1e-12)
    drift[layer] = {"l2": l2, "relative": rel}

results = {
    "train_loss": train_loss, "train_ppl": train_ppl,
    "val_loss": val_loss, "val_ppl": val_ppl,
    "stats_train": stats_train,
    "stats_val": stats_val,
    "activation_drift": drift
}

# JSON SAFE OUTPUT
results = to_python(results)

json.dump(results, open("eval_baseline.json", "w"), indent=2)
print("\n✓ Evaluation complete. Results saved to eval_baseline.json")


Device: cuda
Tokenizer loaded. Vocab size: 275
Train tokens: 207833
Val tokens  : 23093
✓ Model restored from checkpoint.

========== Computing Perplexities ==========
Train Loss: 0.40178808391094206 Train PPL: 1.4944945916669858
Val Loss : 0.4065910530090332 Val PPL : 1.5016898685221982

========== Collecting Layer Stats ==========


100%|██████████| 50/50 [00:58<00:00,  1.16s/it]


========== Computing Drift ==========

✓ Evaluation complete. Results saved to eval_baseline.json


Evaluation Metrics

In [ ]:
import json

with open("eval_baseline.json", "r") as f:
    data = json.load(f)

import pprint
#pprint.pprint(data)
print("Train Loss:", data["train_loss"])
print("Train Perplexity:", data["train_ppl"])
print("Val Loss:", data["val_loss"])
print("Val Perplexity:", data["val_ppl"])



Basline Metrics

In [ ]:
# ----------------------------------
# Baseline Metrics Viewer
# ----------------------------------

import json, torch
from pprint import pprint

with open("eval_baseline.json", "r") as f:
    data = json.load(f)

print("========== BASIC METRICS ==========")
print("Train Loss:", data["train_loss"])
print("Train PPL :", data["train_ppl"])
print("Val Loss  :", data["val_loss"])
print("Val PPL   :", data["val_ppl"])
print("")

# -------------------------------------------------------
# 1. Residual Activation Mean Norms per Layer
# -------------------------------------------------------
print("========== RESIDUAL MEAN NORM (Train) ==========")
for layer, stats in data["residual_stats_train"].items():
    print(f"Layer {layer}: mean_norm = {stats['mean_norm']:.4f}")

print("\n========== RESIDUAL MEAN NORM (Val) ==========")
for layer, stats in data["residual_stats_val"].items():
    print(f"Layer {layer}: mean_norm = {stats['mean_norm']:.4f}")

# -------------------------------------------------------
# 2. Residual Activation STD Norm per Layer
# -------------------------------------------------------
print("\n========== RESIDUAL STD NORM (Train) ==========")
for layer, stats in data["residual_stats_train"].items():
    std_vec = torch.tensor(stats["std"])
    print(f"Layer {layer}: std_norm = {std_vec.norm().item():.4f}")

print("\n========== RESIDUAL STD NORM (Val) ==========")
for layer, stats in data["residual_stats_val"].items():
    std_vec = torch.tensor(stats["std"])
    print(f"Layer {layer}: std_norm = {std_vec.norm().item():.4f}")

# -------------------------------------------------------
# 3. Activation Drift
# -------------------------------------------------------
print("\n========== ACTIVATION DRIFT (Train ↔ Val) ==========")
for layer, drift_vals in data["activation_drift"].items():
    print(f"Layer {layer}: L2={drift_vals['l2']:.4f}, Relative={drift_vals['relative']:.4f}")

print("\nDone viewing all baseline metrics.")
